In [43]:
import numpy as np
import keras
from keras.models import Sequential , Model
from keras.layers import Input , RepeatVector , TimeDistributed  , GRU , Bidirectional , LSTM , Dense
from keras import regularizers
import os
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from keras.models import model_from_json
from keras.datasets import mnist


gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [67]:
def bimodel(maxlen,word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = Bidirectional(GRU(128 , return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(inputs)
    encoded = Bidirectional(GRU(64, return_sequences=False , activity_regularizer=regularizers.l2(0.01)))(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = Bidirectional(GRU(64 ,return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(decoded)
    decoded = Bidirectional(GRU(word_vector ,return_sequences=True , activity_regularizer=regularizers.l2(0.01)))(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [68]:
def origin_model(maxlen , word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = GRU(128 , return_sequences=True)(inputs)
    encoded = GRU(64, return_sequences=False)(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = GRU(64 ,return_sequences=True)(decoded)
    decoded = GRU(word_vector ,return_sequences=True)(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [73]:
def double_model(maxlen , word_vector):
    inputs = Input(shape=(maxlen , word_vector))
    encoded = GRU(256 , return_sequences=True)(inputs)
    encoded = GRU(128, return_sequences=False)(encoded)
    decoded = RepeatVector(maxlen)(encoded)
    decoded = GRU(128,return_sequences=True)(decoded)
    decoded = GRU(word_vector ,return_sequences=True)(decoded)
    output = TimeDistributed(Dense(word_vector))(decoded)
    sequence_autoencoder = Model(inputs, output)
    
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [17]:
def example_model(timesteps , input_dim ,latent_dim):
    inputs = Input(shape=(timesteps, input_dim))
    encoded = LSTM(latent_dim)(inputs)

    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(input_dim, return_sequences=True)(decoded)

    sequence_autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)
    return sequence_autoencoder

In [33]:
(x_train, _), (x_test, _) = mnist.load_data()

In [34]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28))  # 'channels_firtst'이미지 데이터 형식을 사용하는 경우 이를 적용
x_test = np.reshape(x_test, (len(x_test), 28, 28))  # 'ch

In [36]:
x_train.shape

(60000, 28, 28)

In [37]:
test_model = example_model(28,28,64)

In [38]:
test_model.summary()
test_model.compile(optimizer='adadelta', loss='binary_crossentropy')

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                23808     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 28, 64)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 28, 28)            10416     
Total params: 34,224
Trainable params: 34,224
Non-trainable params: 0
_________________________________________________________________


In [40]:
test_model.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/50


43008/60000 [====================>.........] - ETA: 7:04 - loss: 1.394 - ETA: 3:44 - loss: 1.201 - ETA: 2:38 - loss: 1.107 - ETA: 2:04 - loss: 1.017 - ETA: 1:44 - loss: 0.947 - ETA: 1:31 - loss: 0.885 - ETA: 1:21 - loss: 0.836 - ETA: 1:14 - loss: 0.795 - ETA: 1:08 - loss: 0.767 - ETA: 1:04 - loss: 0.740 - ETA: 1:00 - loss: 0.717 - ETA: 57s - loss: 0.696 - ETA: 55s - loss: 0.67 - ETA: 52s - loss: 0.66 - ETA: 51s - loss: 0.64 - ETA: 49s - loss: 0.63 - ETA: 47s - loss: 0.62 - ETA: 46s - loss: 0.61 - ETA: 45s - loss: 0.60 - ETA: 44s - loss: 0.60 - ETA: 43s - loss: 0.59 - ETA: 42s - loss: 0.58 - ETA: 41s - loss: 0.58 - ETA: 40s - loss: 0.57 - ETA: 39s - loss: 0.56 - ETA: 39s - loss: 0.56 - ETA: 38s - loss: 0.55 - ETA: 37s - loss: 0.55 - ETA: 37s - loss: 0.54 - ETA: 36s - loss: 0.54 - ETA: 36s - loss: 0.53 - ETA: 35s - loss: 0.53 - ETA: 35s - loss: 0.52 - ETA: 35s - loss: 0.52 - ETA: 34s - loss: 0.51 - ETA: 34s - loss: 0.51 - ETA: 33s - loss: 0.51 - ETA: 33s - loss: 0.50 - ETA: 33s - loss: 0

60000/60000 [==============================] - ETA: 7s - loss: 0.347 - ETA: 7s - loss: 0.347 - ETA: 7s - loss: 0.347 - ETA: 7s - loss: 0.346 - ETA: 7s - loss: 0.346 - ETA: 7s - loss: 0.346 - ETA: 7s - loss: 0.346 - ETA: 7s - loss: 0.346 - ETA: 7s - loss: 0.345 - ETA: 7s - loss: 0.345 - ETA: 7s - loss: 0.345 - ETA: 7s - loss: 0.345 - ETA: 6s - loss: 0.345 - ETA: 6s - loss: 0.345 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.344 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.343 - ETA: 6s - loss: 0.342 - ETA: 6s - loss: 0.342 - ETA: 6s - loss: 0.342 - ETA: 5s - loss: 0.342 - ETA: 5s - loss: 0.342 - ETA: 5s - loss: 0.342 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0.341 - ETA: 5s - loss: 0

43520/60000 [====================>.........] - ETA: 26s - loss: 0.26 - ETA: 26s - loss: 0.27 - ETA: 26s - loss: 0.27 - ETA: 26s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.28 - ETA: 26s - loss: 0.28 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 25s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 0.27 - ETA: 24s - loss: 

60000/60000 [==============================] - ETA: 7s - loss: 0.272 - ETA: 7s - loss: 0.272 - ETA: 7s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 6s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.271 - ETA: 5s - loss: 0.271 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 5s - loss: 0.272 - ETA: 4s - loss: 0

...

43520/60000 [====================>.........] - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 23s - loss: 0.22 - ETA: 23s - loss: 0.22 - ETA: 23s - loss: 

60000/60000 [==============================] - ETA: 7s - loss: 0.225 - ETA: 7s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 6s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 5s - loss: 0.225 - ETA: 4s - loss: 0.225 - ETA: 4s - loss: 0

43520/60000 [====================>.........] - ETA: 25s - loss: 0.24 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.23 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 25s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 24s - loss: 0.22 - ETA: 23s - loss: 0.22 - ETA: 23s - loss: 0.22 - ETA: 23s - loss: 

60000/60000 [==============================] - ETA: 7s - loss: 0.223 - ETA: 7s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 6s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 5s - loss: 0.223 - ETA: 4s - loss: 0.223 - ETA: 4s - loss: 0

29184/60000 [=============>................] - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 25s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 24s - loss: 0.21 - ETA: 23s - loss: 0.21 - ETA: 23s - loss: 0.21 - ETA: 23s - loss: 

In [53]:
bimodel = bimodel(28,28)
bimodel.compile(optimizer='adam', loss='mae')

In [54]:
bimodel.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 28, 256)           120576    
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 128)               123264    
_________________________________________________________________
repeat_vector_10 (RepeatVect (None, 28, 128)           0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 28, 128)           74112     
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 28, 56)            26376     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 28, 28)            159

In [55]:
bimodel.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 6:27 - loss: 463.126 - ETA: 3:26 - loss: 439.941 - ETA: 2:25 - loss: 417.232 - ETA: 1:55 - loss: 400.292 - ETA: 1:37 - loss: 382.919 - ETA: 1:25 - loss: 367.609 - ETA: 1:16 - loss: 352.773 - ETA: 1:09 - loss: 339.863 - ETA: 1:04 - loss: 327.983 - ETA: 1:00 - loss: 317.131 - ETA: 56s - loss: 306.774 - ETA: 53s - loss: 296.77 - ETA: 50s - loss: 287.70 - ETA: 48s - loss: 279.16 - ETA: 46s - loss: 271.48 - ETA: 44s - loss: 264.09 - ETA: 43s - loss: 257.12 - ETA: 41s - loss: 250.55 - ETA: 40s - loss: 244.43 - ETA: 39s - loss: 238.40 - ETA: 38s - loss: 232.73 - ETA: 37s - loss: 227.35 - ETA: 36s - loss: 222.18 - ETA: 35s - loss: 217.24 - ETA: 34s - loss: 212.50 - ETA: 33s - loss: 208.00 - ETA: 32s - loss: 203.75 - ETA: 32s - loss: 199.68 - ETA: 31s - loss: 195.71 - ETA: 30s - loss: 191.93 - ETA: 30s - loss: 188.30 - ETA: 29s - loss: 184.85 - ETA: 28s - loss: 181.49 - ETA: 28s - lo

60000/60000 [==============================] - ETA: 28s - loss: 4.47 - ETA: 28s - loss: 4.50 - ETA: 28s - loss: 4.47 - ETA: 28s - loss: 4.46 - ETA: 28s - loss: 4.45 - ETA: 27s - loss: 4.43 - ETA: 27s - loss: 4.42 - ETA: 27s - loss: 4.42 - ETA: 27s - loss: 4.39 - ETA: 26s - loss: 4.36 - ETA: 26s - loss: 4.35 - ETA: 26s - loss: 4.33 - ETA: 26s - loss: 4.31 - ETA: 25s - loss: 4.28 - ETA: 25s - loss: 4.26 - ETA: 25s - loss: 4.25 - ETA: 25s - loss: 4.24 - ETA: 24s - loss: 4.23 - ETA: 24s - loss: 4.22 - ETA: 24s - loss: 4.19 - ETA: 24s - loss: 4.18 - ETA: 24s - loss: 4.16 - ETA: 23s - loss: 4.15 - ETA: 23s - loss: 4.13 - ETA: 23s - loss: 4.12 - ETA: 23s - loss: 4.11 - ETA: 22s - loss: 4.09 - ETA: 22s - loss: 4.08 - ETA: 22s - loss: 4.06 - ETA: 22s - loss: 4.05 - ETA: 21s - loss: 4.03 - ETA: 21s - loss: 4.02 - ETA: 21s - loss: 4.00 - ETA: 20s - loss: 3.99 - ETA: 20s - loss: 3.97 - ETA: 20s - loss: 3.96 - ETA: 20s - loss: 3.95 - ETA: 19s - loss: 3.93 - ETA: 19s - loss: 3.92 - ETA: 19s - loss: 

60000/60000 [==============================] - ETA: 28s - loss: 1.39 - ETA: 28s - loss: 1.38 - ETA: 28s - loss: 1.37 - ETA: 28s - loss: 1.37 - ETA: 28s - loss: 1.37 - ETA: 28s - loss: 1.37 - ETA: 27s - loss: 1.36 - ETA: 27s - loss: 1.35 - ETA: 27s - loss: 1.35 - ETA: 26s - loss: 1.35 - ETA: 26s - loss: 1.34 - ETA: 26s - loss: 1.34 - ETA: 26s - loss: 1.34 - ETA: 25s - loss: 1.34 - ETA: 25s - loss: 1.34 - ETA: 25s - loss: 1.33 - ETA: 25s - loss: 1.33 - ETA: 24s - loss: 1.33 - ETA: 24s - loss: 1.33 - ETA: 24s - loss: 1.32 - ETA: 24s - loss: 1.32 - ETA: 23s - loss: 1.32 - ETA: 23s - loss: 1.32 - ETA: 23s - loss: 1.31 - ETA: 23s - loss: 1.31 - ETA: 22s - loss: 1.31 - ETA: 22s - loss: 1.31 - ETA: 22s - loss: 1.30 - ETA: 22s - loss: 1.30 - ETA: 21s - loss: 1.30 - ETA: 21s - loss: 1.30 - ETA: 21s - loss: 1.30 - ETA: 21s - loss: 1.29 - ETA: 20s - loss: 1.29 - ETA: 20s - loss: 1.29 - ETA: 20s - loss: 1.29 - ETA: 20s - loss: 1.28 - ETA: 19s - loss: 1.28 - ETA: 19s - loss: 1.28 - ETA: 19s - loss: 

60000/60000 [==============================] - ETA: 29s - loss: 0.68 - ETA: 29s - loss: 0.69 - ETA: 28s - loss: 0.68 - ETA: 28s - loss: 0.68 - ETA: 28s - loss: 0.68 - ETA: 27s - loss: 0.68 - ETA: 27s - loss: 0.68 - ETA: 27s - loss: 0.67 - ETA: 27s - loss: 0.68 - ETA: 26s - loss: 0.68 - ETA: 26s - loss: 0.67 - ETA: 26s - loss: 0.67 - ETA: 26s - loss: 0.67 - ETA: 26s - loss: 0.67 - ETA: 25s - loss: 0.67 - ETA: 25s - loss: 0.67 - ETA: 25s - loss: 0.67 - ETA: 25s - loss: 0.67 - ETA: 24s - loss: 0.67 - ETA: 24s - loss: 0.67 - ETA: 24s - loss: 0.67 - ETA: 24s - loss: 0.67 - ETA: 23s - loss: 0.67 - ETA: 23s - loss: 0.67 - ETA: 23s - loss: 0.67 - ETA: 22s - loss: 0.67 - ETA: 22s - loss: 0.67 - ETA: 22s - loss: 0.67 - ETA: 22s - loss: 0.66 - ETA: 21s - loss: 0.66 - ETA: 21s - loss: 0.66 - ETA: 21s - loss: 0.66 - ETA: 21s - loss: 0.66 - ETA: 20s - loss: 0.66 - ETA: 20s - loss: 0.66 - ETA: 20s - loss: 0.66 - ETA: 20s - loss: 0.66 - ETA: 19s - loss: 0.66 - ETA: 19s - loss: 0.65 - ETA: 19s - loss: 

60000/60000 [==============================] - ETA: 29s - loss: 0.42 - ETA: 29s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 28s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 27s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 26s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 25s - loss: 0.42 - ETA: 24s - loss: 0.42 - ETA: 24s - loss: 0.42 - ETA: 24s - loss: 0.41 - ETA: 24s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 23s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 22s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 21s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 20s - loss: 0.41 - ETA: 19s - loss: 0.41 - ETA: 19s - loss: 0.41 - ETA: 19s - loss: 

In [70]:
origin_model1 = origin_model(28,28)

origin_model1.compile(optimizer='adam', loss='mae')

In [71]:
origin_model1.summary()

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
gru_33 (GRU)                 (None, 28, 128)           60288     
_________________________________________________________________
gru_34 (GRU)                 (None, 64)                37056     
_________________________________________________________________
repeat_vector_14 (RepeatVect (None, 28, 64)            0         
_________________________________________________________________
gru_35 (GRU)                 (None, 28, 64)            24768     
_________________________________________________________________
gru_36 (GRU)                 (None, 28, 28)            7812      
_________________________________________________________________
time_distributed_8 (TimeDist (None, 28, 28)            812

In [72]:
origin_model1.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 2:51 - loss: 0.147 - ETA: 1:32 - loss: 0.146 - ETA: 1:05 - loss: 0.143 - ETA: 52s - loss: 0.141 - ETA: 44s - loss: 0.14 - ETA: 39s - loss: 0.14 - ETA: 35s - loss: 0.13 - ETA: 32s - loss: 0.13 - ETA: 29s - loss: 0.13 - ETA: 28s - loss: 0.13 - ETA: 26s - loss: 0.13 - ETA: 25s - loss: 0.13 - ETA: 24s - loss: 0.13 - ETA: 22s - loss: 0.13 - ETA: 22s - loss: 0.13 - ETA: 21s - loss: 0.13 - ETA: 20s - loss: 0.13 - ETA: 20s - loss: 0.13 - ETA: 19s - loss: 0.13 - ETA: 18s - loss: 0.13 - ETA: 18s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 16s - loss: 0.13 - ETA: 16s - loss: 0.13 - ETA: 15s - loss: 0.13 - ETA: 15s - loss: 0.13 - ETA: 15s - loss: 0.13 - ETA: 14s - loss: 0.13 - ETA: 14s - loss: 0.13 - ETA: 14s - loss: 0.13 - ETA: 14s - loss: 0.13 - ETA: 13s - loss: 0.13 - ETA: 13s - loss: 0.13 - ETA: 13s - loss: 0.13 - ETA: 13s - loss: 0.13

60000/60000 [==============================] - ETA: 14s - loss: 0.13 - ETA: 14s - loss: 0.13 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 9s - loss: 0.1279 - ETA: 9s - loss: 

60000/60000 [==============================] - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 14s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 13s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 12s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 11s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 10s - loss: 0.11 - ETA: 9s - loss: 0.1160 - ETA: 9s - loss: 

60000/60000 [==============================] - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 9s - loss: 0

60000/60000 [==============================] - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 14s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 13s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 12s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 11s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 10s - loss: 0.09 - ETA: 9s - loss: 0

In [74]:
double_model = double_model(28,28)
double_model.compile(optimizer='adam', loss='mae')

In [75]:
double_model.summary()

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
gru_37 (GRU)                 (None, 28, 256)           218880    
_________________________________________________________________
gru_38 (GRU)                 (None, 128)               147840    
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 28, 128)           0         
_________________________________________________________________
gru_39 (GRU)                 (None, 28, 128)           98688     
_________________________________________________________________
gru_40 (GRU)                 (None, 28, 28)            13188     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 28, 28)            812

In [76]:
double_model.fit(x_train, x_train,
                epochs=10,
                batch_size=512,
                shuffle=True,
                validation_data=(x_test, x_test),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - ETA: 3:30 - loss: 0.142 - ETA: 1:56 - loss: 0.144 - ETA: 1:23 - loss: 0.145 - ETA: 1:06 - loss: 0.144 - ETA: 56s - loss: 0.142 - ETA: 49s - loss: 0.14 - ETA: 44s - loss: 0.14 - ETA: 40s - loss: 0.14 - ETA: 37s - loss: 0.14 - ETA: 35s - loss: 0.13 - ETA: 32s - loss: 0.13 - ETA: 31s - loss: 0.13 - ETA: 29s - loss: 0.13 - ETA: 28s - loss: 0.13 - ETA: 27s - loss: 0.13 - ETA: 26s - loss: 0.13 - ETA: 25s - loss: 0.13 - ETA: 24s - loss: 0.13 - ETA: 23s - loss: 0.13 - ETA: 22s - loss: 0.13 - ETA: 22s - loss: 0.13 - ETA: 21s - loss: 0.13 - ETA: 20s - loss: 0.13 - ETA: 20s - loss: 0.13 - ETA: 19s - loss: 0.13 - ETA: 19s - loss: 0.13 - ETA: 18s - loss: 0.13 - ETA: 18s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 17s - loss: 0.13 - ETA: 16s - loss: 0.13 - ETA: 16s - loss: 0.13 - ETA: 16s - loss: 0.13 - ETA: 15s - loss: 0.13 - ETA: 15s - loss: 0.13 - ETA: 15s - loss: 0.

60000/60000 [==============================] - ETA: 15s - loss: 0.12 - ETA: 15s - loss: 0.12 - ETA: 15s - loss: 0.12 - ETA: 15s - loss: 0.12 - ETA: 15s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 14s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 13s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 12s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 11s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 0.12 - ETA: 10s - loss: 

60000/60000 [==============================] - ETA: 15s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 15s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 14s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 13s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 12s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 11s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 0.10 - ETA: 10s - loss: 

60000/60000 [==============================] - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 14s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 13s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 12s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 11s - loss: 0.08 - ETA: 10s - loss: 0.08 - ETA: 10s - loss: 0.08 - ETA: 10s - loss: 0.08 - ETA: 10s - loss: 0.08 - ETA: 10s - loss: 

60000/60000 [==============================] - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.08 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 15s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 14s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 13s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 12s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 11s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 0.07 - ETA: 10s - loss: 